In [1]:
%pwd

'/Users/rohithkamath/Documents/endtoendproject/Chicken-Disease-Classification/research'

In [2]:
import os


In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rohithkamath/Documents/endtoendproject/Chicken-Disease-Classification'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [19]:
from clasifier_cnn.constants import *
from clasifier_cnn.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir  = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)]
        )

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

        

In [20]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time



In [28]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_checkpoint_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]


In [29]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_checkpoint_callbacks()
    
except Exception as e:
    raise e


[2025-05-05 20:01:04,327: INFO: common_func: Yaml file: config/config.yaml has been loaded succesfully]
[2025-05-05 20:01:04,331: INFO: common_func: Yaml file: params.yaml has been loaded succesfully]
[2025-05-05 20:01:04,332: INFO: common_func: created directory at: artifacts]
[2025-05-05 20:01:04,333: INFO: common_func: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-05-05 20:01:04,334: INFO: common_func: created directory at: artifacts/prepare_callbacks/tensorflow_log_dir]
